# Goal


In this exercise, you will carry out a geometry optimization of an [endohedral fullerene](https://en.wikipedia.org/wiki/Endohedral_fullerene). These are carbon cages that contain atoms, ions, or molecular clusters in their inner spheres.

For more information the options available to you, feel free to refer to the [Structure Optimization](https://ase-lib.org/ase/optimize.html) section of the ASE documentation.


# Setup


In [23]:
%uv pip install ase@git+https://gitlab.com/ase/ase.git 'matcalc[matgl]'

Using Python 3.13.7 environment at: /Users/ct5868/personal/cbe423/.venv
   Updating https://gitlab.com/ase/ase.git (HEAD)                  
   Updating https://gitlab.com/ase/ase.git (HEAD)          
   Updating https://gitlab.com/ase/ase.git (HEAD)          
   Updating https://gitlab.com/ase/ase.git (HEAD)          
    Updated https://gitlab.com/ase/ase.git (9e022fa5b8d123bc1785145661104a4f893a
Resolved 75 packages in 592ms                                        
Audited 75 packages in 0.83ms
Note: you may need to restart the kernel to use updated packages.


# Picking a Fullerene


In [24]:
import os
import urllib.request

url = "https://nanotube.msu.edu/fullerene/C60/C60-Ih.xyz"
filename = url.split("/")[-1]

if not os.path.exists(filename):
    urllib.request.urlretrieve(url, filename)
    print("Downloaded.")
else:
    print("File already exists, skipping download.")

Downloaded.


First, we need to start with a fullerene structure. Several fullerene isomers can be found at [this link](https://nanotube.msu.edu/fullerene/fullerene-isomers.html). Pick a fullerene of your choice and download the corresponding `.xyz` file, which contains the atomic coordinates. The choice is yours, but you probably don't want to go larger than C320 or so for the sake of obtaining speedy results.


Once you have downloaded the `.xyz` file for your chosen fullerene, visualize it using the ASE GUI. This can be done via `ase gui <filename.xyz>` in the command line or using the following code in Python.

```python
from ase.io import read
from ase.visualize import view

atoms = read("filename.xyz")
view(atoms)
```

It is also possible to view the `.xyz` in VESTA, but we will want to manipulate the structure, which is hard to do in VESTA. [SAMSON](https://www.samson-connect.net/) is a nice program to do this if you prefer something a bit fancier than ASE for the visualization.


In [30]:
%uv run ase gui {filename}

Note: you may need to restart the kernel to use updated packages.


# Making an Endohedral Fullerene


With your chosen fullerene, add a species to the inside of the sphere. Pick any atom or small molecule that you would like, provided it reasonably fits inside the cage.

This can be done using a GUI or programmatically (e.g. by adding two `Atoms` objects together). If you are using the ASE GUI, you can save a new `.xyz` file with `File > Save`.


# Geometry Optimization


Since you placed the atom or molecule in the fullerene somewhat arbitrarily, the structure you have generated is not a local minimum in the potential energy landscape. In other words, a lower energy configuration can be found than what you guessed.

Using ASE with the TensorNet-MatPES-r2SCAN machine learning interatomic potential, carry out a geometry optimization on your endohedral fullerene.

Always make sure your results are sensible:

- Look at the energy vs. step info: does the energy look like it converged?
- Visualize the trajectory (`.traj`): does the geometry optimization process look sensible?
- Investigate the final structure: does it look chemically reasonable?


In [31]:
from matcalc import load_fp

calc = load_fp("TensorNet-MatPES-r2SCAN-v2025.1-PES")

In [35]:
from ase.io import read

atoms = read(filename)  # your .xyz file
atoms.calc = calc
original_atoms = atoms.copy()

In [36]:
from ase.optimize import BFGS

BFGS(atoms).run(fmax=0.05)

      Step     Time          Energy          fmax
BFGS:    0 14:28:10     -557.191772        2.315723
BFGS:    1 14:28:10     -558.862671        1.206914
BFGS:    2 14:28:10     -559.755310        0.927845
BFGS:    3 14:28:10     -560.995911        0.326103
BFGS:    4 14:28:10     -561.011475        0.120084
BFGS:    5 14:28:10     -561.013794        0.070786
BFGS:    6 14:28:10     -561.014832        0.049287


np.True_

# Vibrational Analysis


Generally, it is good practice to confirm that the structure you found is indeed a minimum energy structure. This can be done by carrying out a vibrational frequency analysis and confirming that all the vibrational modes are real, positive numbers.

This can be expensive and sometimes altogether computationally prohibitive, because requires carrying out 3N single-point calculations corresponding to the displacements in +/- x, y, and z for each, where N is the number of atoms in your system.

For a molecule, 6 of the vibrational modes (or 5 if the molecule is linear) should be ignored. These modes are actually translations and rotations rather than true vibrations. They are generally small values close to 0. After removing these modes, the remainder should be real and positive if you are a minimum. Some codes will remove these spurious vibrational modes automatically, but ASE's `Vibrations` module does not.

For additional details on carrying out vibrational analysis in ASE, refer to the [corresponding documentation](https://ase-lib.org/ase/vibrations/vibrations.html). Note that for periodic solids, it is common to carry out a [phonon calculation](https://ase-lib.org/ase/phonons.html) to determine that the structure is "dynamically stable," but this is beyond the scope of the current demonstration.


In [37]:
from ase.vibrations import Vibrations


vib = Vibrations(atoms)  # replace `final_atoms`
vib.clean()  # make sure we start fresh
vib.run()  # run vibrational analysis
vib.summary()  # report results

---------------------
  #    meV     cm^-1
---------------------
  0    0.2i      1.3i
  1    0.1i      0.6i
  2    0.0i      0.3i
  3    0.1       0.8
  4    0.4       3.4
  5    0.7       5.8
  6   30.7     247.7
  7   30.7     247.8
  8   30.7     247.9
  9   30.8     248.2
 10   30.8     248.3
 11   43.3     349.6
 12   43.4     350.1
 13   43.4     350.4
 14   44.0     354.6
 15   44.0     354.9
 16   44.0     355.0
 17   44.0     355.3
 18   46.5     374.8
 19   46.5     374.8
 20   46.5     374.8
 21   46.5     374.8
 22   46.5     374.8
 23   54.8     442.1
 24   54.8     442.3
 25   54.9     442.5
 26   54.9     442.6
 27   54.9     442.8
 28   57.8     466.5
 29   57.9     466.7
 30   57.9     466.8
 31   57.9     467.0
 32   60.4     486.8
 33   64.0     516.0
 34   64.0     516.2
 35   64.0     516.2
 36   64.0     516.5
 37   64.1     516.7
 38   67.8     547.0
 39   67.8     547.1
 40   67.9     547.3
 41   69.9     563.9
 42   69.9     563.9
 43   69.9     564.0
 44   69